# QOSF Mentorship Program - Screening Task

## Submitted By : Smit Chaudhary

### Task Chosen : Task 2

Implement a circuit that returns state $|01\rangle$ and state $|10\rangle$ with equal probability (50% each).

**Requirements**

- The circuit should consist only of CNOTs, RXs and RYs.
- Start from all parameters in parametric gates being equal to 0 or randomly chosen.
- You should find the right set of parameters using gradient descent (you can use more advanced optimization methods if you like).
- Simulations must be done with sampling (i.e. a limited number of measurements per iteration) and noise.

Compare the results for different numbers of measurements: 1, 10, 100, 1000.

### Analyse the Problem

Firstly, we notice that the task of getting one of the bell state is a task that is routinely performed for many algorithms and protocols such as *Super-dense Coding*, *Quantum Teleportation* etc. Thus, it is a fairly well known task.

Consider the following circuit:

<img src = '1.png'>

*Screenshot taken from [Quantum-Inspie](https://www.quantum-inspire.com/)*

This simple circuit with 2 gates would give us a bell state. Though notably, one would not get an equal superposition of $|01\rangle$ and $|10\rangle$ as we wanted but an equal superposition of $|00\rangle$ and $|11\rangle$.

Though we can flip any of the two qubit either at the beginning or at the end to get an equal superposition of $|01\rangle$ and $|10\rangle$.

Now, having established the fact that it is a fairly simply procedure, we notice that in our task, we are not allowed to use the hadamard gate and only allowed to use the parameteric $R_x$, $R_y$ gates and the CNOT gate.

Thus, we will have to decompose the X gate and the Hadamard gate in terms of these gates to be able to solve our problem.

Firstly, X gate is same as rotating the qubits around Y axis by $\pi$.

Then, we see that the Hadamard gate is a combination of rotation about X axis by $\pi/2$ and rotation about Y by $\pi/2$.

For the CNOT gate, we directly apply the CNOT gate since we are allowed to use it.

But, in the problem, we are not allowed to directly use $\pi$ or $\pi2$. Thus, we will initialise the parametric gates $R_x$ and $R_y$ with parameters (which are the angles of rotation) being equal to 0. We will construct a cost function and try to minimize it using gradient descent to find the optimal values of the parameters.

### Using Gradient Descent
It is an iterative optimization algorithm to find local minima. We need to be careful with the parameters (of gradient descent, and not the function) we choose while applying gradient descent.

Here, we take a `function_to_be_minimized` and find the values of the parameters for which it is minimum.

We need to choose certain parameters of the gradient descent.

Learning rate = 0.8  
Step size = 0.005  
Maximum iterations of gradient descent = 1000

Note that we have maximum iterations of gradient descent set at 1000 since we don't want our iterative process to continue indefinitely.

We need to ensure Learning rate and Step Size are not too big or we could miss the minima altogether.

To start, we import some useful libraries and define our system.

In [213]:
import numpy as np
import random

from qiskit import(Aer, QuantumCircuit, execute)
import qiskit.providers.aer.noise as noise

import matplotlib.pyplot as plt

simulator = Aer.get_backend('qasm_simulator')
backend = Aer.get_backend('statevector_simulator')

In [223]:
number_of_measurements = [1, 10, 100, 1000]

In [215]:
learning_rate = 0.8
step_size = 0.005
max_gd_iter = 1000

Now, we are ready to apply gradient descent and find the optimal values of parameters but to help facilitate that, let us define a couple of functions first. These are the sub-tasks that we need to perform repeatedly so let us define these functions.

In [216]:
def get_counts(circuit, no_of_measurements):
    
    circuit.measure_all()
    result = execute(circuit, simulator, shots=no_of_measurements, noise_model=noise.NoiseModel()).result()
    counts  = result.get_counts(circuit)
    return counts

def probability(count, no_of_measurements):
    
    probability_0 = 0
    probability_1 = 0
    
    if '0' in count.keys():
        probability_0 = int(count['0'])/no_of_measurements
    if '1' in count.keys():
        probability_1 = int(count['1'])/no_of_measurements  
        
    return  probability_0, probability_1

These are two fairly simple functions whose names are self explantory. The function `get_counts` measures and gives us the counts for each measurement state. Similarly, the function`probability` gives us the probability based on those counts.

Similarly, but more importantly, we now break down the gradient descent process in smaller functions. The description of functions follows.

In [217]:
def calculate_derivative(theta_index, function_to_be_minimised, theta, function_value, no_of_measurements):

    new_theta = theta.copy()
    
    new_theta[theta_index] = theta[theta_index] + step_size
    new_function_value = function_to_be_minimised(new_theta, no_of_measurements)
    
    derivative = (new_function_value - function_value)/step_size
    return derivative

def update_theta(theta_index, theta, gradient):
    
    theta[theta_index] -= learning_rate*gradient[theta_index]
    theta[theta_index] =  theta[theta_index] % (2*np.pi)
    
    return theta[theta_index]

def gradient_descent(initial_theta, function_to_be_minimised, no_of_measurements): 
    
    theta = np.array(initial_theta).copy()
    dimensions_of_theta = len(initial_theta)
    
    min_theta = np.array(theta).copy()
    function_value = function_to_be_minimised(theta, no_of_measurements)
    min_function_value = function_value 
    
    for iteration in range(max_gd_iter): 
       
       gradient = []
       for theta_index in range(dimensions_of_theta):
          deriv = calculate_derivative(theta_index, function_to_be_minimised, theta, function_value, no_of_measurements)
          gradient.append(deriv)

       for theta_index in range(dimensions_of_theta): 
          theta[theta_index] = update_theta(theta_index, theta, gradient)
    
       function_value = function_to_be_minimised(theta, no_of_measurements)
       
       if (function_value == 0): 
           min_function_value = function_value
           min_theta = np.array(theta).copy()
        
       if (min_function_value > function_value):
           min_function_value = function_value
           min_theta = np.array(theta).copy()
         
    print("Minimum value of the function: ", function_value)
    print("Value of theta for minnima of function: ", min_theta)
    return min_function_value, min_theta

`calculate_derivative` is a function that calculates the derivative of the function with respect to whatever parameter is given. Note that we call it derivative as of now and not the gradient since when the function's support spans more than 1 dimensions, gradient would be array of these derivatives.

`update_theta` function updates the value of `theta` (which could be an array) based on the value of the gradient. We want each $\theta \in [0,2\pi]$. So we take modulo $2\pi$.

Finally, `gradient_descent` performs the entire gradient descent algorithm and returns a tuple which has the minimum value of the function that is to be minimised and also the value of the input parameters at which this minima is achieved.

### Finding theta for X Gate

Now, since we have all our tools in aresenal, let us find the value of theta such that we get $X$ gate from the parametric gate.

Note that if we had an X gate and applied on state $|0\rangle$ we would get $|1\rangle$ and upon measurement, the probability of getting state $|0\rangle$ is 0 and probability of getting state $|1\rangle$ is 1.

Thus, we can define the error function, which is to be minimised as
$$f = (prob(0) - 0)^2 + (prob(1) - 1)^2$$

In [226]:
def function_for_X(theta, no_of_measurements):
    circuit = QuantumCircuit(1)
    
    # X gate on qubit 1
    circuit.ry(theta[0], 0)
    
    count = get_counts(circuit, no_of_measurements)
    probability_0, probability_1 = probability(count, no_of_measurements)
 
    function_val = ((float(probability_0) - 0)**2 + (float(probability_1) - 1)**2)
    return function_val

initial_theta = [random.uniform(0, 2*np.pi)]

X_results = {}

for n in number_of_measurements:
    print()
    print("For", n, "measurement(s):")
    
    min_function_value, min_theta = gradient_descent(initial_theta, function_for_X, n)
    X_results[n] = [min_function_value, min_theta]


For 1 measurement(s):
Minimum value of the function:  0.0
Value of theta for minnima of function:  [3.16465808]

For 10 measurement(s):
Minimum value of the function:  0.0
Value of theta for minnima of function:  [3.11865478]

For 100 measurement(s):
Minimum value of the function:  0.0
Value of theta for minnima of function:  [3.1376197]

For 1000 measurement(s):
Minimum value of the function:  0.0
Value of theta for minnima of function:  [3.15345808]


### Finding theta for H Gate

Now, since we have all our tools in aresenal, let us find the value of theta such that we get $H$ gate from the parametric gates.

Note that if we had an H gate and applied on state $|0\rangle$ we would get $|+\rangle$ and upon measurement, the probability of getting state $|0\rangle$ is 0.5 and probability of getting state $|1\rangle$ is 0.5

Thus, we can define the error function, which is to be minimised as
$$f = (prob(0) - 0.5)^2 + (prob(1) - 0.5)^2$$

In [228]:
def function_for_H(theta, no_of_measurements):
    circuit = QuantumCircuit(1)
    
    circuit.ry(theta[0],0)
    circuit.rx(theta[1],0)

    count = get_counts(circuit, no_of_measurements)
    probability_0, probability_1 = probability(count, no_of_measurements)
    
    function_val = ((float(probability_0) - 0.5)**2 + (float(probability_1) - 0.5)**2)
    return function_val

initial_theta = [random.uniform(0,2*np.pi), random.uniform(0,2*np.pi)]

H_results = {}

for n in number_of_measurements:
    print()
    print("For", n, "measurement(s):")
    
    min_function_value, min_theta = gradient_descent(initial_theta, function_for_H, n)
    H_results[n] = [min_function_value, min_theta]


For 1 measurement(s):
Minimum value of the function:  0.5
Value of theta for minnima of function:  [4.37827078 4.54520318]

For 10 measurement(s):
Minimum value of the function:  0.0
Value of theta for minnima of function:  [4.93753614 4.03194111]

For 100 measurement(s):
Minimum value of the function:  0.5
Value of theta for minnima of function:  [4.76071486 3.40283256]

For 1000 measurement(s):
Minimum value of the function:  0.014791999999999996
Value of theta for minnima of function:  [4.78500547 1.43736318]


## Analysing the Results

This would be a good time to look at our results and comment on some interesting artefacts. Which are interesting in themselves but also help us understand the intricacies of the process and help us solve the bonus part of the question.

### The curious case of H gate

If we go by the names of variables and functions, it seems like using gradient descent we have found the parameters which would help us administer a Hadamard Gate but that is not true. The function that we are minimising dictates that we get a final state such that the probability of getting $|0\rangle$ and $|1\rangle$ are 0.5 each. This could happen in many different ways and we need not apply exactly the Hadamard Gate to get these values. Note that we would want both the angles to be $\pi/2$ to get Hadamard. But even if we applied a gate that took $|0\rangle$ to $|-\rangle$ instead of $|+\rangle$ we would still get equal probability and the error function woudl be minimum there.

And since we randomly initialise our parameters, we are not sure where exactly the function would converge to.

### But does it really work?

Let us apply the parametric circuit on a system and check the results. Note, that the third part is applying a CNOT gate which we are allowed to and we don't need to optimize anything for it.

To do this, let us define a function that gives us probabilities, similar to the one we defined earlier but this time for 2 qubits instead of just 1.

In [229]:
def probability_2_qubits(count, no_of_measurements):
    
    probability_00 = 0
    probability_01 = 0
    probability_10 = 0
    probability_11 = 0
    
    if '00' in count.keys():
        probability_00 = int(count['00'])/no_of_measurements
    if '01' in count.keys():
        probability_01 = int(count['01'])/no_of_measurements
    if '10' in count.keys():
        probability_10 = int(count['10'])/no_of_measurements
    if '11' in count.keys():
        probability_11 = int(count['11'])/no_of_measurements
        
    return  probability_00, probability_01, probability_10, probability_11

We now check the results using a function that first applies all the gates using the parameters we arrived at and then returns the counts for each state along with the function value.

In [230]:
def check_results(X_results, H_results):
    
    counts = []
    func_values = []
    
    for n in number_of_measurements:    
        circuit = QuantumCircuit(2)

        circuit.ry(X_results[n][1][0],1)

        circuit.ry(H_results[n][1][0],0)
        circuit.rx(H_results[n][1][1],0)

        circuit.cx(0,1)
        
        count = get_results(circuit, n)
        counts.append(count)
        
        probability_00, probability_01, probability_10, probability_11 = probability_2_qubits(count, n)
        
        function_val = ((float(probability_00) - 0)**2 + (float(probability_01) - 0.5)**2 + (float(probability_10) - 0.5)**2 + (float(probability_11) - 0)**2)
        func_values.append(function_val)         
    return counts, func_values
        
counts, function_values = check_results(X_results, H_results)

Wow! Nothing happened. Not yet. We can now check these results and compare them to what we expect for an ideal X and H gate.

In [231]:
for i in range(4):
    probability_00 = 0
    probability_01 = 0
    probability_10 = 0
    probability_11 = 0
    
    print("for ", number_of_measurements[i], "measurements, we have:")
    if '00' in counts[i].keys():
        probability_00 = float(counts[i]['00'])/number_of_measurements[i]
    print("probability of getting 00 is ", probability_00)
    if '01' in counts[i].keys():
        probability_01 = float(counts[i]['01'])/number_of_measurements[i]
    print("probability of getting 01 is ", probability_01)
    if '10' in counts[i].keys():
        probability_10 = float(counts[i]['10'])/number_of_measurements[i]
    print("probability of getting 10 is ", probability_10)
    if '11' in counts[i].keys():
        probability_11 = float(counts[i]['11'])/number_of_measurements[i]
    print("probability of getting 11 is ", probability_11)
    print()

for  1 measurements, we have:
probability of getting 00 is  0
probability of getting 01 is  1.0
probability of getting 10 is  0
probability of getting 11 is  0

for  10 measurements, we have:
probability of getting 00 is  0
probability of getting 01 is  0.4
probability of getting 10 is  0.6
probability of getting 11 is  0

for  100 measurements, we have:
probability of getting 00 is  0
probability of getting 01 is  0.53
probability of getting 10 is  0.47
probability of getting 11 is  0

for  1000 measurements, we have:
probability of getting 00 is  0
probability of getting 01 is  0.482
probability of getting 10 is  0.518
probability of getting 11 is  0



Wow! This looks neat but why is the probability not close to 0.5 for 1 measurement? It is simply becasue if we have one measurement, we get either 01 or 10. And then when we calculate probability based on our observation, we will get probability 1 for either of them.

This also ties in to how as the number of measurements increase, the error reduces. We find the maximum error for 1 measurement and the least for 1000 measurements. For n measurements where n is large enough, we would find the probability to be 0.5 for both the states.

## Bonus Question

You thought we were done? Not yet. But almost. We are going to take a shortcut here. We can have Qiskit provide us the state our qubit(s) are in and then can simply see if we have the right state to check if we are getting $\frac{1}{\sqrt{2}}\left(|01\rangle + |10\rangle\right)$ or some other state which gives us these two states with probability 0.5

We are going to ensure that this time we aactually apply the Hadamard gate and not anything else. A hadamard gate would take $|0\rangle$ to $\frac{1}{\sqrt{2}}\left(|0\rangle + |1\rangle\right)$

Thus, just as before, we construct an error function that we have to minimize. The function this time will be:
$$f = (real(|0\rangle) - \frac{1}{\sqrt{2}})^2 + (real(|1\rangle) - \frac{1}{\sqrt{2}})^2$$

For this, let us define a function that returns us the 

In [233]:
def function_for_H(theta, no_of_measurements):
    circuit = QuantumCircuit(1)
    
    circuit.ry(theta[0],0)
    circuit.rx(theta[1],0)
    
    state = execute(circuit, backend).result().get_statevector()
    function_val = ((float(state[0].real) - (1/2)**(1/2))**2 + (float(state[1].real) - (1/2)**(1/2))**2)
    return function_val

initial_theta = [random.uniform(0,2*np.pi), random.uniform(0,2*np.pi)]
    
min_function_value, min_theta = gradient_descent(initial_theta, function_for_H, 1000)

Minimum value of the function:  1.724734169765496e-08
Value of theta for minnima of function:  [1.57079819 4.59713454]


In [234]:
def checking_state(X_results, min_theta):   
    circuit = QuantumCircuit(2)

    # X gate on qubit 1
    circuit.ry(X_results[n][1][0],1)

    # H gate on qubit 0
    circuit.ry(min_theta[0],0)
    circuit.rx(min_theta[1],0)

    # CNOT gate with qubit 0 as a control qubit and qubit 1 as the targer qubit
    circuit.cx(0,1)
    state_vector = get_state_vector(circuit)
        
    return state_vector
        
state_vector = checking_state(X_results, min_theta)
print(state_vector)

[-0.00419504+0.00000000e+00j  0.70709426-1.30538304e-06j
  0.70709441+0.00000000e+00j -0.00419504+7.74455274e-09j]


## WE ARE DONE

The state is exactly what we desired.